One problem with the automatic fiducial detection is that it is very slow for large fields of view (it can take hours). The slow down comes from the linking across frames. This is problematic because one may need to try different parameters to achieve good fiducial detection.

A possible solution is to visually select regions containing fiducials from histogram plots. Then, the linking can be made to occur only over the search region.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


### Clean up the data

In [2]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV1_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file)

/home/kmdouglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
stringMask             = df['uncertainty [nm]'].map(lambda x: isinstance(x, str)).as_matrix()
df['uncertainty [nm]'] = pd.to_numeric(df['uncertainty [nm]'], errors='coerce')
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna()
df.reindex()
print('Done.')

Done.


# Display 2D histograms
2D histograms seem to be a good way to identify fiducials, especially in microtubules and the centriole samples. This is because the fiducials don't drift much and should be present in every frame, leading to large values in the bins that hide the other features.

Let's first try to generate a nice looking 2D histogram that tells us the information we need, specifically the location of the fiducials.

In [4]:
toMicrons = .001 # Conversion of columns to microns

fig, ax = plt.subplots()
im      = ax.hexbin(df['x [nm]'] * toMicrons, df['y [nm]'] * toMicrons, gridsize = 100, cmap = plt.cm.YlOrRd_r)
ax.set_xlabel(r'x-position, $\mu m$')
ax.set_ylabel(r'y-position, $\mu m$')
ax.invert_yaxis()

cb      = plt.colorbar(im)
cb.set_label('Counts')

plt.show()

In [5]:
plt.close()

# Making interactive rectangular selections
Now that we have a decent looking histogram that can identify fiducials, let's try making an interactive, rectangular selection widget. This code is based on the [matplotlib 1.5 docs](http://matplotlib.org/api/widgets_api.html) and the [rectangle selector example](http://matplotlib.org/examples/widgets/rectangle_selector.html).

In [4]:
from matplotlib.widgets import RectangleSelector

def onselect(eclick, erelease):
    pass
    '''# eclick and e release are matplotlib events at press and release
    print(' startposition : (%f, %f)' % (eclick.xdata, eclick.ydata))
    print(' endposition   : (%f, %f)' % (erelease.xdata, erelease.ydata))
    print(' used button   : ', eclick.button)'''

def toggle_selector(event):
    print(' Key pressed.')
    if event.key in ['Q', 'q'] and toggle_selector.RS.active:
        print(' RectangleSelector deactivated.')
        toggle_selector.RS.set_active(False)
    if event.key in ['A', 'a'] and not toggle_selector.RS.active:
        print(' RectangleSelector activated.')
        toggle_selector.RS.set_active(True)
    if event.key in [' ']:
        print('Space bar pressed!')
        
toMicrons = .001 # Conversion of columns to microns

fig, ax = plt.subplots()
im      = ax.hexbin(df['x [nm]'] * toMicrons, df['y [nm]'] * toMicrons, gridsize = 100, cmap = plt.cm.YlOrRd_r)
ax.set_xlabel(r'x-position, $\mu m$')
ax.set_ylabel(r'y-position, $\mu m$')
ax.invert_yaxis()

cb      = plt.colorbar(im)
cb.set_label('Counts')

toggle_selector.RS = RectangleSelector(ax,
                                       onselect,
                                       drawtype    = 'box',
                                       useblit     = True,
                                       button      = [1, 3],  # don't use middle button
                                       spancoords  = 'data',
                                       interactive = True)
plt.connect('key_press_event', toggle_selector)
plt.show()

In [5]:
plt.close()

In [6]:
toggle_selector.RS.extents

(1.1904941838942271,
 3.3875377356673457,
 32.830574988541656,
 35.100628688281233)